In [0]:
display(spark.read.table("samples.nyctaxi.trips"))

In [0]:
data = [[1, "VW"],  
        [2, "BMW"]] 

columns = ["ID", "Car"] 

dataframe = spark.createDataFrame(data, columns) 
  
# show data frame 
display(dataframe)

ID,Car
1,VW
2,BMW


In [0]:
data = [
[1,"Alicia Kaiser","WA","001-677-774-4370","13714","Stacy Summit","54799","Lake Joseborough","Delaware","1939-04-07","2656508165701512", "1000.00"],
[2,"Donna Ellis","BR","001-631-995-8008","43599","Adam Trail","07204","Port April","Montana","1966-07-28","5527331190171381",  "1000.00"],
[3,"Kenneth Smith","WA","001-592-849-6009x4173","649","Sherri Grove","14527","North Miranda","Washington","1998-09-25","5366314062583069",  "1000.00"],
[4,"Danny Clark","WA","574.419.0221x348","285","Timothy Drive","41106","West Erica","Maryland","1948-08-29","5488489084734990",  "2000.00"],
[5,"Nicholas Thompson","CA","(259)268-8760x061","998","Russell Shoals","65647","South Todd","South Carolina","1985-03-08","2720282150775392", "3000.00"],
[6,"Frances Griffith","WA","7535316823","9559","Emily Branch","71422","Mcdanielhaven","New York","1984-07-12","2248334835679706", "4000.00"],
[7,"Trevor Harrington","CA","742.224.9375","5960","Lisa Port","73881","Loganbury","New York","1979-03-10","5130498353342015", "2000.00"],
[8,"Seth Mitchell","AA","(386)517-7589x04440","47352","Stafford Loop","01347","South Alexander","North Dakota","1944-07-03","2280935548220544", "3000.00"],
[9,"Patrick Caldwell","BR","001-307-225-9094","0170","Amanda Dam","24885","Port Mollyhaven","Connecticut","1973-05-04","5557276831443314", "2000.00"],
[10,"Laura Hopkins","CA","9095819755","143","Lee Brook","23623","Jarvisland","Hawaii","1971-11-13","2720224762678291", "100.00"]
]
schama = "client_number int,name string,branch string,phone_number string,bulding_number string,street_name string,postcode string,city string,state string,birth_date string,credit_card_number string, amout string"
df = spark.createDataFrame(data, schama)

In [0]:
display(df.select("client_number","name", "amout"))

client_number,name,amout
1,Alicia Kaiser,1000.00
2,Donna Ellis,1000.00
3,Kenneth Smith,1000.00
4,Danny Clark,2000.00
5,Nicholas Thompson,3000.00
6,Frances Griffith,4000.00
7,Trevor Harrington,2000.00
8,Seth Mitchell,3000.00
9,Patrick Caldwell,2000.00
10,Laura Hopkins,100.00


In [0]:
df.createTempView("client")

In [0]:
%sql
select client_number,amout from client

client_number,amout
1,1000.00
2,1000.00
3,1000.00
4,2000.00
5,3000.00
6,4000.00
7,2000.00
8,3000.00
9,2000.00
10,100.00


There are various options to access columns in a DataFrame. Below, you can find a few examples:

In [0]:
from pyspark.sql.functions import expr, col, column
display(df.select(
expr("client_number as client_num"),
col("client_number").alias("client_num"),
column("client_number"))\
.limit(2))

client_num,client_num,client_number
1,1,1
2,2,2


During data transformation, common scenarios include changing data types, renaming columns, adding new columns, and deriving new columns based on values from others. Let’s explore how we can achieve these tasks.

In [0]:
from pyspark.sql.functions import expr, col, column, lit, exp, current_date

age_exp = "extract( year from current_date) - extract( year from birth_date) "

df1 = df.withColumn("amout", col("amout").cast("decimal(10,2)")) \
     .withColumn("birth_date", col("birth_date").cast("date")) \
     .withColumn("age", expr(age_exp)) \
     .withColumn("load_date", lit(current_date()))

df1 = df1.drop("credit_card_number", "birth_date")

display(df1)

# In this example, we utilize the withColumn function to calculate age, add load date columns, and change the amount to decimal data type. Additionally, we can drop columns with unnecessary data. The same result can be achieved using SQL.

client_number,name,branch,phone_number,bulding_number,street_name,postcode,city,state,amout,age,load_date
1,Alicia Kaiser,WA,001-677-774-4370,13714,Stacy Summit,54799,Lake Joseborough,Delaware,1000.00,86,2025-03-14
2,Donna Ellis,BR,001-631-995-8008,43599,Adam Trail,07204,Port April,Montana,1000.00,59,2025-03-14
3,Kenneth Smith,WA,001-592-849-6009x4173,649,Sherri Grove,14527,North Miranda,Washington,1000.00,27,2025-03-14
4,Danny Clark,WA,574.419.0221x348,285,Timothy Drive,41106,West Erica,Maryland,2000.00,77,2025-03-14
5,Nicholas Thompson,CA,(259)268-8760x061,998,Russell Shoals,65647,South Todd,South Carolina,3000.00,40,2025-03-14
6,Frances Griffith,WA,7535316823,9559,Emily Branch,71422,Mcdanielhaven,New York,4000.00,41,2025-03-14
7,Trevor Harrington,CA,742.224.9375,5960,Lisa Port,73881,Loganbury,New York,2000.00,46,2025-03-14
8,Seth Mitchell,AA,(386)517-7589x04440,47352,Stafford Loop,01347,South Alexander,North Dakota,3000.00,81,2025-03-14
9,Patrick Caldwell,BR,001-307-225-9094,0170,Amanda Dam,24885,Port Mollyhaven,Connecticut,2000.00,52,2025-03-14
10,Laura Hopkins,CA,9095819755,143,Lee Brook,23623,Jarvisland,Hawaii,100.00,54,2025-03-14


In [0]:
df.createOrReplaceTempView("client")

df1 = spark.sql("""
               select 
                client_number,
                name,
                bulding_number,
                street_name,
                postcode,
                city,
                state,
                cast(amout as decimal(10,2)) amount,
                extract( year from current_date) - extract( year from birth_date) as age,
                current_date()  load_date
               from 
               client
               """)
display(df1)   

client_number,name,bulding_number,street_name,postcode,city,state,amount,age,load_date
1,Alicia Kaiser,13714,Stacy Summit,54799,Lake Joseborough,Delaware,1000.00,86,2025-03-14
2,Donna Ellis,43599,Adam Trail,07204,Port April,Montana,1000.00,59,2025-03-14
3,Kenneth Smith,649,Sherri Grove,14527,North Miranda,Washington,1000.00,27,2025-03-14
4,Danny Clark,285,Timothy Drive,41106,West Erica,Maryland,2000.00,77,2025-03-14
5,Nicholas Thompson,998,Russell Shoals,65647,South Todd,South Carolina,3000.00,40,2025-03-14
6,Frances Griffith,9559,Emily Branch,71422,Mcdanielhaven,New York,4000.00,41,2025-03-14
7,Trevor Harrington,5960,Lisa Port,73881,Loganbury,New York,2000.00,46,2025-03-14
8,Seth Mitchell,47352,Stafford Loop,01347,South Alexander,North Dakota,3000.00,81,2025-03-14
9,Patrick Caldwell,0170,Amanda Dam,24885,Port Mollyhaven,Connecticut,2000.00,52,2025-03-14
10,Laura Hopkins,143,Lee Brook,23623,Jarvisland,Hawaii,100.00,54,2025-03-14


Filtering Rows in a DataFrame
Filtering rows in a DataFrame involves creating a condition that separates the data you want to keep from the data you don’t. This condition can be written as a simple expression or built from multiple comparisons. DataFrames offer two methods, where and filter, to achieve this filtering based on your chosen condition.

In [0]:
display(df1.where("age >= 85"))


client_number,name,bulding_number,street_name,postcode,city,state,amount,age,load_date
1,Alicia Kaiser,13714,Stacy Summit,54799,Lake Joseborough,Delaware,1000.00,86,2025-03-14


In [0]:
display(df1.where(col("age") >= 85))

client_number,name,bulding_number,street_name,postcode,city,state,amount,age,load_date
1,Alicia Kaiser,13714,Stacy Summit,54799,Lake Joseborough,Delaware,1000.00,86,2025-03-14


iltering using AND or OR conditions, allowing for greater flexibility in specifying conditions for row selection.

In [0]:
display(df1.where("age >= 85 or age <=30"))
display(df1.filter( (col("age") >= 85) |  (col("age") <= 30)))

client_number,name,bulding_number,street_name,postcode,city,state,amount,age,load_date
1,Alicia Kaiser,13714,Stacy Summit,54799,Lake Joseborough,Delaware,1000.00,86,2025-03-14
3,Kenneth Smith,649,Sherri Grove,14527,North Miranda,Washington,1000.00,27,2025-03-14


client_number,name,bulding_number,street_name,postcode,city,state,amount,age,load_date
1,Alicia Kaiser,13714,Stacy Summit,54799,Lake Joseborough,Delaware,1000.00,86,2025-03-14
3,Kenneth Smith,649,Sherri Grove,14527,North Miranda,Washington,1000.00,27,2025-03-14


In [0]:
display(df1.where("age >=30 and branch != 'WA' "))
display(df1.filter( (col("age") >= 30) &  (col("branch") != 'WA')))

client_number,name,bulding_number,street_name,postcode,city,state,amount,age,load_date
2,Donna Ellis,43599,Adam Trail,07204,Port April,Montana,1000.00,59,2025-03-14
5,Nicholas Thompson,998,Russell Shoals,65647,South Todd,South Carolina,3000.00,40,2025-03-14
7,Trevor Harrington,5960,Lisa Port,73881,Loganbury,New York,2000.00,46,2025-03-14
8,Seth Mitchell,47352,Stafford Loop,01347,South Alexander,North Dakota,3000.00,81,2025-03-14
9,Patrick Caldwell,0170,Amanda Dam,24885,Port Mollyhaven,Connecticut,2000.00,52,2025-03-14
10,Laura Hopkins,143,Lee Brook,23623,Jarvisland,Hawaii,100.00,54,2025-03-14


client_number,name,bulding_number,street_name,postcode,city,state,amount,age,load_date
2,Donna Ellis,43599,Adam Trail,07204,Port April,Montana,1000.00,59,2025-03-14
5,Nicholas Thompson,998,Russell Shoals,65647,South Todd,South Carolina,3000.00,40,2025-03-14
7,Trevor Harrington,5960,Lisa Port,73881,Loganbury,New York,2000.00,46,2025-03-14
8,Seth Mitchell,47352,Stafford Loop,01347,South Alexander,North Dakota,3000.00,81,2025-03-14
9,Patrick Caldwell,0170,Amanda Dam,24885,Port Mollyhaven,Connecticut,2000.00,52,2025-03-14
10,Laura Hopkins,143,Lee Brook,23623,Jarvisland,Hawaii,100.00,54,2025-03-14


Grouping and Sorting
Group by is a transformation operation in PySpark used to group the data in a Spark DataFrame based on specified columns. This operation is often followed by aggregating functions such as count(), sum(), avg(), etc., allowing for the summarization of grouped data.

Order by, on the other hand, sorts records in a DataFrame based on specified sort conditions, allowing for the arrangement of data in ascending or descending order.

In [0]:
from pyspark.sql.functions import desc, count, sum

# count by brnahc
display(df1.groupBy("city").count().orderBy(desc("count")))
    

city,count
Port April,1
Lake Joseborough,1
South Todd,1
West Erica,1
Loganbury,1
Jarvisland,1
Port Mollyhaven,1
North Miranda,1
Mcdanielhaven,1
South Alexander,1


In [0]:
# sum, count by branch with order by client_number count
display(df1.groupBy("state").agg( 
sum("amount").alias("state_amount"), count("client_number").alias("noofclients") 
).orderBy(desc("noofclients")))

state,state_amount,noofclients
New York,6000.00,2
Delaware,1000.00,1
Washington,1000.00,1
North Dakota,3000.00,1
Montana,1000.00,1
Maryland,2000.00,1
South Carolina,3000.00,1
Hawaii,100.00,1
Connecticut,2000.00,1


In [0]:
df.createOrReplaceTempView("client")
df1 = spark.sql("""
               select
                state,
                sum(amout) amount,
                count(client_number) noofclient_number

               from
               client
               group by state
               """)

df1.show()

+--------------+------+-----------------+
|         state|amount|noofclient_number|
+--------------+------+-----------------+
|       Montana|1000.0|                1|
|      Maryland|2000.0|                1|
|South Carolina|3000.0|                1|
|      New York|6000.0|                2|
|        Hawaii| 100.0|                1|
|   Connecticut|2000.0|                1|
|      Delaware|1000.0|                1|
|    Washington|1000.0|                1|
|  North Dakota|3000.0|                1|
+--------------+------+-----------------+



Joining DataFrames
Joining operations are crucial for various data processing tasks such as data normalization, data modeling, and ensuring data quality. Spark supports joins using DataFrame join and SQL joins.

To demonstrate the join operation, we need an additional DataFrame. I’ll create it similarly so you can easily replicate my steps, or you can load data from a file or table.

In [0]:
import uuid
trans = [
    [str(uuid.uuid4()), 1, 100.00, '2024-02-01'],
    [str(uuid.uuid4()), 1, 200.00, '2024-02-03'],
    [str(uuid.uuid4()), 1, 130.00, '2024-02-04'],
    [str(uuid.uuid4()), 2, 110.00, '2024-02-05'],
    [str(uuid.uuid4()), 3, 200.00, '2024-02-01'],
    [str(uuid.uuid4()), 2, 300.00, '2024-02-02'],
    [str(uuid.uuid4()), 2, 50.00, '2024-02-03'],

]

schama = "id string, client_id int, value double, tran_date string"
df_tran = spark.createDataFrame(trans, schama)
display(df_tran)

id,client_id,value,tran_date
fb0bc3f9-61d8-44d6-b9bb-ee9279c9020c,1,100.0,2024-02-01
26076af0-d8dc-46ff-bb2c-aece0703ee21,1,200.0,2024-02-03
a71b6a75-c5e5-429c-8946-8df996d0657e,1,130.0,2024-02-04
a7fa903a-0e61-437c-b7f9-1987c834260a,2,110.0,2024-02-05
59661f8e-6cb2-403d-9b28-d3eac5c97375,3,200.0,2024-02-01
9aa2c49f-3f26-42b6-b63c-518f0f06711a,2,300.0,2024-02-02
86ddb0bf-d020-4803-8035-c35ae320cfac,2,50.0,2024-02-03


The code below illustrates how to join two DataFrames. Spark supports various join types, including:

Inner Join
Left / Left Outer Join
Right / Right Outer Join
Outer / Full Join
Cross Join
Left Anti Join
Left Semi Join

In [0]:
# column name index style
display(df.join(df_tran, df['client_number'] == df_tran['client_id'], 'inner'))

# column name property style
display(df.join(df_tran, df.client_number == df_tran.client_id, 'inner'))

client_number,name,branch,phone_number,bulding_number,street_name,postcode,city,state,birth_date,credit_card_number,amout,id,client_id,value,tran_date
1,Alicia Kaiser,WA,001-677-774-4370,13714,Stacy Summit,54799,Lake Joseborough,Delaware,1939-04-07,2656508165701512,1000.00,fb0bc3f9-61d8-44d6-b9bb-ee9279c9020c,1,100.0,2024-02-01
1,Alicia Kaiser,WA,001-677-774-4370,13714,Stacy Summit,54799,Lake Joseborough,Delaware,1939-04-07,2656508165701512,1000.00,a71b6a75-c5e5-429c-8946-8df996d0657e,1,130.0,2024-02-04
1,Alicia Kaiser,WA,001-677-774-4370,13714,Stacy Summit,54799,Lake Joseborough,Delaware,1939-04-07,2656508165701512,1000.00,26076af0-d8dc-46ff-bb2c-aece0703ee21,1,200.0,2024-02-03
2,Donna Ellis,BR,001-631-995-8008,43599,Adam Trail,07204,Port April,Montana,1966-07-28,5527331190171381,1000.00,86ddb0bf-d020-4803-8035-c35ae320cfac,2,50.0,2024-02-03
2,Donna Ellis,BR,001-631-995-8008,43599,Adam Trail,07204,Port April,Montana,1966-07-28,5527331190171381,1000.00,a7fa903a-0e61-437c-b7f9-1987c834260a,2,110.0,2024-02-05
2,Donna Ellis,BR,001-631-995-8008,43599,Adam Trail,07204,Port April,Montana,1966-07-28,5527331190171381,1000.00,9aa2c49f-3f26-42b6-b63c-518f0f06711a,2,300.0,2024-02-02
3,Kenneth Smith,WA,001-592-849-6009x4173,649,Sherri Grove,14527,North Miranda,Washington,1998-09-25,5366314062583069,1000.00,59661f8e-6cb2-403d-9b28-d3eac5c97375,3,200.0,2024-02-01


client_number,name,branch,phone_number,bulding_number,street_name,postcode,city,state,birth_date,credit_card_number,amout,id,client_id,value,tran_date
1,Alicia Kaiser,WA,001-677-774-4370,13714,Stacy Summit,54799,Lake Joseborough,Delaware,1939-04-07,2656508165701512,1000.00,fb0bc3f9-61d8-44d6-b9bb-ee9279c9020c,1,100.0,2024-02-01
1,Alicia Kaiser,WA,001-677-774-4370,13714,Stacy Summit,54799,Lake Joseborough,Delaware,1939-04-07,2656508165701512,1000.00,26076af0-d8dc-46ff-bb2c-aece0703ee21,1,200.0,2024-02-03
1,Alicia Kaiser,WA,001-677-774-4370,13714,Stacy Summit,54799,Lake Joseborough,Delaware,1939-04-07,2656508165701512,1000.00,a71b6a75-c5e5-429c-8946-8df996d0657e,1,130.0,2024-02-04
2,Donna Ellis,BR,001-631-995-8008,43599,Adam Trail,07204,Port April,Montana,1966-07-28,5527331190171381,1000.00,9aa2c49f-3f26-42b6-b63c-518f0f06711a,2,300.0,2024-02-02
2,Donna Ellis,BR,001-631-995-8008,43599,Adam Trail,07204,Port April,Montana,1966-07-28,5527331190171381,1000.00,86ddb0bf-d020-4803-8035-c35ae320cfac,2,50.0,2024-02-03
2,Donna Ellis,BR,001-631-995-8008,43599,Adam Trail,07204,Port April,Montana,1966-07-28,5527331190171381,1000.00,a7fa903a-0e61-437c-b7f9-1987c834260a,2,110.0,2024-02-05
3,Kenneth Smith,WA,001-592-849-6009x4173,649,Sherri Grove,14527,North Miranda,Washington,1998-09-25,5366314062583069,1000.00,59661f8e-6cb2-403d-9b28-d3eac5c97375,3,200.0,2024-02-01


In [0]:
# - Left Anti join: Retrieves records from the left DataFrame that do not exist in the right DataFrame.
display(df.join(df_tran, df['client_number'] == df_tran['client_id'], 'left_anti').select("client_number"))


client_number
4
5
6
7
8
9
10


In [0]:
# Left Semi-join: Retrieves records and columns from the left DataFrame where records match records in the right DataFrame.
display(df.join(df_tran, df['client_number'] == df_tran['client_id'], 'leftsemi'))

client_number,name,branch,phone_number,bulding_number,street_name,postcode,city,state,birth_date,credit_card_number,amout
1,Alicia Kaiser,WA,001-677-774-4370,13714,Stacy Summit,54799,Lake Joseborough,Delaware,1939-04-07,2656508165701512,1000.00
2,Donna Ellis,BR,001-631-995-8008,43599,Adam Trail,07204,Port April,Montana,1966-07-28,5527331190171381,1000.00
3,Kenneth Smith,WA,001-592-849-6009x4173,649,Sherri Grove,14527,North Miranda,Washington,1998-09-25,5366314062583069,1000.00


In [0]:
# Naturally, Spark SQL allows us to use SQL-like joins in our code. We can join existing tables or create views and join these views using SQL syntax.

df.createOrReplaceTempView("clients")
df_tran.createOrReplaceTempView("tran")

display(spark.sql("""
select * 
from 
clients a 
inner join 
tran b on a.client_number = b.client_id
                  """))


client_number,name,branch,phone_number,bulding_number,street_name,postcode,city,state,birth_date,credit_card_number,amout,id,client_id,value,tran_date
1,Alicia Kaiser,WA,001-677-774-4370,13714,Stacy Summit,54799,Lake Joseborough,Delaware,1939-04-07,2656508165701512,1000.00,26076af0-d8dc-46ff-bb2c-aece0703ee21,1,200.0,2024-02-03
1,Alicia Kaiser,WA,001-677-774-4370,13714,Stacy Summit,54799,Lake Joseborough,Delaware,1939-04-07,2656508165701512,1000.00,fb0bc3f9-61d8-44d6-b9bb-ee9279c9020c,1,100.0,2024-02-01
1,Alicia Kaiser,WA,001-677-774-4370,13714,Stacy Summit,54799,Lake Joseborough,Delaware,1939-04-07,2656508165701512,1000.00,a71b6a75-c5e5-429c-8946-8df996d0657e,1,130.0,2024-02-04
2,Donna Ellis,BR,001-631-995-8008,43599,Adam Trail,07204,Port April,Montana,1966-07-28,5527331190171381,1000.00,a7fa903a-0e61-437c-b7f9-1987c834260a,2,110.0,2024-02-05
2,Donna Ellis,BR,001-631-995-8008,43599,Adam Trail,07204,Port April,Montana,1966-07-28,5527331190171381,1000.00,9aa2c49f-3f26-42b6-b63c-518f0f06711a,2,300.0,2024-02-02
2,Donna Ellis,BR,001-631-995-8008,43599,Adam Trail,07204,Port April,Montana,1966-07-28,5527331190171381,1000.00,86ddb0bf-d020-4803-8035-c35ae320cfac,2,50.0,2024-02-03
3,Kenneth Smith,WA,001-592-849-6009x4173,649,Sherri Grove,14527,North Miranda,Washington,1998-09-25,5366314062583069,1000.00,59661f8e-6cb2-403d-9b28-d3eac5c97375,3,200.0,2024-02-01


In [0]:
# Union DataFrames
df1 = spark.createDataFrame([[1, 2, 3]], ["col0", "col1", "col2"])

df2 = spark.createDataFrame([[4, 5, 6]], ["col1", "col2", "col0"])

# example 1
display(df1.unionByName(df2))

# example 2
display(df1.union(df2))

df2 = spark.createDataFrame([[4, 5, 6]], ["col1", "col2", "col3"])

# example 3
display(df1.unionByName(df2, allowMissingColumns=True))

col0,col1,col2
1,2,3
6,4,5


col0,col1,col2
1,2,3
4,5,6


col0,col1,col2,col3
1,2,3,null
null,4,5,6


In [0]:
#When function
#A useful function in PySpark is when, which is employed in cases where we need to translate or map a value to another value based on specified conditions


from pyspark.sql.functions import expr, col, column, lit, exp, current_date, when

data = [("Robert", "Smith","M",40000),("Ana","Novak","M",60000),
        ("Carl","Xyz",None,500000),("Maria","Free","F",500000),
        ("Johm","Az","",None), ("Steve","Smith","",1000)]

columns = ["name","surname","gender","salary"]
df = spark.createDataFrame(data = data, schema = columns)

df2 = df.withColumn("new_gender", when(df.gender == "M","Male")
                                 .when(df.gender == "F","Female")
                                 .when(df.gender.isNull() ,"")
                                 .otherwise(df.gender))
display(df)
display(df2)

name,surname,gender,salary
Robert,Smith,M,40000
Ana,Novak,M,60000
Carl,Xyz,null,500000
Maria,Free,F,500000
Johm,Az,,null
Steve,Smith,,1000


name,surname,gender,salary,new_gender
Robert,Smith,M,40000,Male
Ana,Novak,M,60000,Male
Carl,Xyz,null,500000,
Maria,Free,F,500000,Female
Johm,Az,,null,
Steve,Smith,,1000,


Python Functions and Modules

In [0]:
# fun.py
def add(a,b):
    return a+b

User-defined functions
If we can’t find a function for our use case, it’s possible to create our custom function. However, to use it with DataFrames or SQL, we need to register it.

In [0]:
%python
from pyspark.sql.types import LongType

def squared(s):
  return s * s

spark.udf.register("squaredWithPython", squared, LongType())
df1.createOrReplaceTempView("df1")

def add(a, b):
  return a + b

print(add(1, 2))

3


In [0]:
import requests

response = requests.get('https://health.data.ny.gov/api/views/jxy9-yhdk/rows.csv')
csvfile = response.content.decode('utf-8')
dbutils.fs.put("/Volumes/main/default/my-volume/babynames.csv", csvfile, True)

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-3750198211146947>, line 5
      3 response = requests.get('https://health.data.ny.gov/api/views/jxy9-yhdk/rows.csv')
      4 csvfile = response.content.decode('utf-8')
----> 5 dbutils.fs.put("/Volumes/main/default/my-volume/babynames.csv", csvfile, True)

File /databricks/python_shell/lib/dbruntime/dbutils.py:158, in prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    156 exc.__context__ = None
    157 exc.__cause__ = None
--> 158 raise exc

ExecutionError: An error occurred while calling o412.put.
: com.databricks.backend.daemon.data.common.InvalidMountException: Error while using path /Volumes/main/default/my-volume for creating file system within mount at '/Volumes/main/default/my-volume'.
	at com.databricks.backend.daemon.data.common.InvalidMountException$.apply(DataMessages.scala:765)
	a

In [0]:
babynames = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/Volumes/main/default/my-volume/babynames.csv")
babynames.createOrReplaceTempView("babynames_table")
years = spark.sql("select distinct(Year) from babynames_table").toPandas()['Year'].tolist()
years.sort()
dbutils.widgets.dropdown("year", "2014", [str(x) for x in years])
display(babynames.filter(babynames.Year == dbutils.widgets.get("year")))

Reading and Writing from and to Azure Data Lake Gen2

Unity Catalog